In [1]:
from scipy.io import loadmat
import numpy as np
import pandas as pd



In [6]:
def calculate_energy_baseline(year):

    pv = loadmat(f'data/PV_{year}.mat')

    pv_gen = pv['data_2'][12]
    time = pv['data_2'][0].astype('int')

    pv_df_hour = pd.DataFrame(index=time, data=pv_gen, columns=['pv_gen']) 
    pv_df_hour['day'] = pv_df_hour.index//(3600*24)

    pv_df_day = 2.0*14/14*pv_df_hour.groupby('day').sum()[:-1]
    

    columns = ['FLEXLAB-XR-ZONEB LIGHTS:Lights Electric Power [W](TimeStep)',
               'FLEXLAB-XR-ZONEB ELECEQ:Electric Equipment Electric Power [W](TimeStep)',
               'SYS-B HEATING COIL:Heating Coil Heating Rate [W](TimeStep)',
               'SYS-B COOLING COIL:Cooling Coil Total Cooling Rate [W](TimeStep)',
               'SYS-B SUPPLY FAN:Fan Electric Power [W](TimeStep)',
               'SYS-B CW SEC CIRC PUMP2:Pump Electric Power [W](TimeStep)',
               'SYS-B HW SEC CIRC PUMP2:Pump Electric Power [W](TimeStep)']

    load_hour = pd.read_csv(f'data/{year}.csv',usecols=columns)
    load_hour.columns = ['lig','mels','hea','coo','fan','cooPump','heaPump']

    load_hour['day'] = load_hour.index//(4*24)

    load_day = load_hour.groupby('day').sum()/4  # divided by 4 as each measurement is for 15 mins only

    load_day['total_load'] = load_day[['lig','mels','fan','cooPump','heaPump']].sum(axis=1)+load_day['hea']/0.95+load_day['coo']/3.0
    
    EMax = 6300

    daily_net = np.maximum(load_day['total_load']-pv_df_day['pv_gen'],np.ones(365)*EMax*-1)

    energy_baseline = round(daily_net.sum()/1000,2)
    
    print(f'Energy baseline for year {year} is: {energy_baseline} kWh')

In [7]:
for year in [2013,2014,2015,2017,'TMY']:
    calculate_energy_baseline(year)

Energy baseline for year 2013 is: 40.81 kWh
Energy baseline for year 2014 is: -747.16 kWh
Energy baseline for year 2015 is: -504.17 kWh
Energy baseline for year 2017 is: 220.0 kWh
Energy baseline for year TMY is: -120.66 kWh


In [42]:
for year in [2013,2014,2015,2017,'TMY']:
    calculate_energy_baseline(year)

Energy baseline for year 2013 is: 1814.3 kWh
Energy baseline for year 2014 is: 751.38 kWh
Energy baseline for year 2015 is: 1256.22 kWh
Energy baseline for year 2017 is: 2072.65 kWh
Energy baseline for year TMY is: 1781.52 kWh


In [12]:
7579-2284+415

5710

In [4]:
pv = loadmat('data/PV_2013.mat')

In [5]:
pv_gen = pv['data_2'][12]

In [6]:
2*pv_gen.sum()/1000*(11.0/14.0)

4921.278214285714

In [8]:
year = 2014

columns = ['FLEXLAB-XR-ZONEB LIGHTS:Lights Electric Power [W](TimeStep)',
               'FLEXLAB-XR-ZONEB ELECEQ:Electric Equipment Electric Power [W](TimeStep)',
               'SYS-B HEATING COIL:Heating Coil Heating Rate [W](TimeStep)',
               'SYS-B COOLING COIL:Cooling Coil Total Cooling Rate [W](TimeStep)',
               'SYS-B SUPPLY FAN:Fan Electric Power [W](TimeStep)',
               'SYS-B CW SEC CIRC PUMP2:Pump Electric Power [W](TimeStep)',
               'SYS-B HW SEC CIRC PUMP2:Pump Electric Power [W](TimeStep)']

load_hour = pd.read_csv(f'data/{year}.csv',usecols=columns)
load_hour.columns = ['lig','mels','hea','coo','fan','cooPump','heaPump']

In [9]:
total_lig = load_hour['lig'].sum()*.25/1000
total_mels = load_hour['mels'].sum()*.25/1000
total_hea = load_hour['hea'].sum()*.25/1000
total_coo = load_hour['coo'].sum()*.25/1000
total_fan = load_hour['fan'].sum()*.25/1000
total =total_lig+total_mels+total_hea/.95+total_coo/3.0+total_fan

print('Total Lighting: {0} kWh'.format(total_lig))
print('Total plug load: {0} kWh'.format(total_mels))
print('Total heating: {0} kWh'.format(total_hea/.95))
print('Total cooling: {0} kWh'.format(total_coo/3.0))
print('Total fan: {0} kWh'.format(total_fan))
print('Total energy consumed : {0} kWh'.format(total))

Total Lighting: 1646.389410216 kWh
Total plug load: 1802.1102832989002 kWh
Total heating: 117.50658867582001 kWh
Total cooling: 1149.699721643369 kWh
Total fan: 932.4232616159998 kWh
Total energy consumed : 5648.129265450089 kWh


In [14]:
pv = loadmat(f'data/PV_{year}.mat')

pv_gen = pv['data_2'][12]
time = pv['data_2'][0].astype('int')

pv_df_hour = pd.DataFrame(index=time, data=pv_gen, columns=['pv_gen']) 
pv_df_hour['day'] = pv_df_hour.index//(3600*24)

pv_df_day = pv_df_hour.groupby('day').sum()[:-1]

In [66]:
pv_df_day.sum()/1000

pv_gen    3790.835938
dtype: float32

In [70]:
load_hour['day'] = load_hour.index//(4*24)
load_day = load_hour.groupby('day').sum()/4  # divided by 4 as each measurement is for 15 mins only
load_day['total_load'] = load_day[['lig','mels','fan','cooPump','heaPump']].sum(axis=1)+load_day['hea']/0.95+load_day['coo']/3.0

In [79]:
np.mean(load_day['total_load']*.7-pv_df_day['pv_gen']*10.0/7.0)

-3982.25456099285

In [83]:
np.std(load_day['total_load']*.7-pv_df_day['pv_gen']*10.0/7.0)

TypeError: 'float' object is not subscriptable

In [86]:
diff = load_day['total_load']*.7-pv_df_day['pv_gen']*7.0/7.0

In [87]:
diff[100:150]

day
100    -927.214328
101   -5760.259911
102   -6139.837826
103   -2207.278820
104   -4088.439148
105   -3369.578215
106   -3692.187048
107   -3301.763833
108   -6279.971466
109   -5968.265730
110   -1688.357113
111   -4410.286790
112   -3809.685924
113   -1279.402179
114     489.476460
115   -4361.423723
116   -6327.780687
117   -4545.137648
118   -2189.959798
119    -677.138320
120     -13.803800
121   -4507.353933
122   -5794.613113
123   -3901.827918
124   -4519.424650
125   -5184.383860
126   -3986.414924
127     262.702128
128   -5309.912972
129   -7264.230380
130   -7027.957794
131   -2837.871200
132   -2250.832927
133    -754.627117
134    -639.924746
135   -4203.200474
136   -4399.497173
137   -7114.549452
138   -3007.715803
139   -4432.915453
140   -5653.302223
141   -4417.502262
142   -3966.872806
143   -6609.013227
144   -7600.530608
145   -5554.994465
146   -6464.024561
147   -5947.522878
148   -5059.788903
149   -4127.794506
dtype: float64